In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import preprocess_train as p
from sklearn.decomposition import PCA

In [2]:
iter_csv = pd.read_csv('data/train.csv', iterator=True, chunksize=20000)
#df = pd.concat([chunk[chunk['field'] > constant] for chunk in iter_csv])

In [3]:
df = next(iter_csv)

C:\Users\Matthias\AppData\Local\conda\conda\envs\dl\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
# checking class balance
Y_COLUMN = ["HasDetections"]
colname = Y_COLUMN[0]

df.HasDetections.unique()
#check smallest class:
df[Y_COLUMN].groupby(Y_COLUMN)[Y_COLUMN].count()

#
#data.groupby(['col1', 'col2'])['col3'].mean()

,HasDetections
HasDetections,
0,9980
1,10020


In [27]:
#df.head(5)

# delete columns with not enough values

In [28]:
# delete columns that have not enough values, aka less than 75% of total # of rows
dft = p.delete_sparse_columns(df, p=0.75)

DELETED COLUMNS WITH TOO LESS VALUES: <class 'pandas.core.frame.DataFrame'> - (20000, 74)



In [31]:
df.describe()

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,GeoNameIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
count,20000.0,19929.000000,20000.000000,19917.000000,19917.000000,19917.000000,20000.000000,20000.000000,19260.000000,20000.000000,...,19581.000000,19627.000000,20000.000000,19967.000000,20000.00000,20000.000000,19837.000000,19350.000000,19350.000000,20000.000000
mean,0.0,6.853731,0.016000,47865.443290,1.320982,1.019481,0.988200,108.669600,81445.604517,170.097800,...,399.675910,32837.969277,0.485800,0.006761,0.12465,0.038000,0.057015,0.279225,7.840413,0.501000
std,0.0,0.996447,0.125478,13992.590881,0.518340,0.164119,0.107988,62.867544,48985.435340,88.905986,...,221.553863,21156.270985,0.499811,0.081950,0.33033,0.191201,0.231876,0.448630,4.570912,0.500012
min,0.0,0.000000,0.000000,39.000000,1.000000,0.000000,0.000000,1.000000,41.000000,1.000000,...,17.000000,19.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.0,7.000000,0.000000,49480.000000,1.000000,1.000000,1.000000,51.000000,36825.000000,89.000000,...,142.000000,12951.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,3.000000,0.000000
50%,0.0,7.000000,0.000000,53447.000000,1.000000,1.000000,1.000000,97.000000,82373.000000,181.000000,...,500.000000,33066.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,10.000000,1.000000
75%,0.0,7.000000,0.000000,53447.000000,2.000000,1.000000,1.000000,162.000000,124633.000000,264.000000,...,556.000000,52369.000000,1.000000,0.000000,0.00000,0.000000,0.000000,1.000000,11.000000,1.000000
max,0.0,8.000000,1.000000,70262.000000,5.000000,4.000000,1.000000,222.000000,167907.000000,295.000000,...,1081.000000,72074.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,15.000000,1.000000


# analyze outliers

### clipping

In [40]:
col = "CityIdentifier"
minPercentile = 0.02
maxPercentile = 0.98
q = df["CityIdentifier"].quantile(maxPercentile)
q

164345.05472

In [39]:
# clipping function for one column
df[col] = df[col].clip((df[col].quantile(minPercentile)),(df[col].quantile(maxPercentile)))

In [42]:
df.shape

(20000, 74)

### Z-score

In [59]:
df = p.factorization_encoding(df)

FACTORIZATION ENCODING: <class 'pandas.core.frame.DataFrame'> - (20000, 83)



In [78]:
np.abs(stats.zscore(df) < 3).all(axis = 1)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:2253: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.


array([False, False, False, ..., False, False, False])

In [55]:
from scipy import stats
import numpy as np
df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]


C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:2253: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in less
  This is separate from the ipykernel package so we can avoid doing imports until


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections


In [134]:
#df = pd.DataFrame(np.random.randn(100, 3))
df2 = df[["HasDetections","IsBeta","AVProductStatesIdentifier"]]
df2 = df[["AVProductsInstalled", "AVProductStatesIdentifier","HasDetections", "IsBeta"]]
df2 = p.factorization_encoding(df2)
df2 = p.impute_numerical_NaNs(df2, "mean")
df2.shape
df2.info()

FACTORIZATION ENCODING: <class 'pandas.core.frame.DataFrame'> - (20000, 4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 40000 to 59999
Data columns (total 4 columns):
AVProductsInstalled          20000 non-null float64
AVProductStatesIdentifier    20000 non-null float64
HasDetections                20000 non-null float64
IsBeta                       20000 non-null float64
dtypes: float64(4)
memory usage: 625.1 KB


D:\datascience\projects\ml_pipeline\preprocess_train.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  imp.fit(columndata)
D:\datascience\projects\ml_pipeline\preprocess_train.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  imp.fit(columndata)
D:\datascience\projects\ml_pipeline\preprocess_train.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/ind

In [135]:
df2[(np.abs(stats.zscore(df2)) < 3).all(axis=1)].shape

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:2253: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in less
  """Entry point for launching an IPython kernel.


(0, 4)

In [106]:
df2.shape

(20000, 3)

In [121]:
df3 = pd.DataFrame(np.random.randn(100, 3))
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
0    100 non-null float64
1    100 non-null float64
2    100 non-null float64
dtypes: float64(3)
memory usage: 2.4 KB


In [124]:
df3[(np.abs(stats.zscore(df3)) < 3).all(axis=1)].shape

(100, 3)

# Analyzing numerical features

In [29]:
df.groupby("HasDetections").mean()
#---> make percentage of max-min from max --> relevance of the differences!

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,GeoNameIdentifier,...,Census_IsFlightsDisabled,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
HasDetections,,,,,,,,,,,,,,,,,,,,,
0,0.0,6.818758,0.018938,46276.995169,1.396678,1.025969,0.988477,107.923046,81814.993749,169.566333,...,0.0,404.262645,32841.403437,0.488677,0.011035,0.140681,0.038978,0.070434,0.256503,7.889244
1,0.0,6.888511,0.013074,49446.412242,1.245642,1.013023,0.987924,109.413174,81078.585861,170.627146,...,0.0,395.129754,32834.561263,0.482934,0.002500,0.108683,0.037026,0.043618,0.302008,7.791451


In [30]:
df.groupby("HasDetections").mean().std()

IsBeta                                               0.000000e+00
RtpStateBitfield                                     4.932256e-02
IsSxsPassiveMode                                     4.146491e-03
AVProductStatesIdentifier                            2.241116e+03
AVProductsInstalled                                  1.067988e-01
AVProductsEnabled                                    9.153748e-03
HasTpm                                               3.908902e-04
CountryIdentifier                                    1.053679e+00
CityIdentifier                                       5.207190e+02
GeoNameIdentifier                                    7.501081e-01
LocaleEnglishNameIdentifier                          2.460149e+00
OsBuild                                              8.240077e+01
OsSuite                                              1.327820e+01
IsProtected                                          1.835588e-02
AutoSampleOptIn                                      0.000000e+00
SMode     

# create dummies

In [20]:
object_columns = list(df.select_dtypes(include=['object']))
df_encoded = pd.get_dummies(df, prefix=object_columns)
df_encoded.shape
df_encoded.head(5)

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,GeoNameIdentifier,...,Census_ActivationChannel_Retail:TB:Eval,Census_ActivationChannel_Volume:GVLK,Census_ActivationChannel_Volume:MAK,Census_FlightRing_Disabled,Census_FlightRing_NOT_SET,Census_FlightRing_RP,Census_FlightRing_Retail,Census_FlightRing_Unknown,Census_FlightRing_WIF,Census_FlightRing_WIS
0,0,7.0,0,53447.0,1.0,1.0,1,29,128035.0,35,...,0,0,0,0,0,0,1,0,0,0
1,0,7.0,0,53447.0,1.0,1.0,1,93,1482.0,119,...,0,0,0,0,1,0,0,0,0,0
2,0,7.0,0,53447.0,1.0,1.0,1,86,153579.0,64,...,0,0,0,0,0,0,1,0,0,0
3,0,7.0,0,53447.0,1.0,1.0,1,88,20710.0,117,...,0,0,0,0,0,0,1,0,0,0
4,0,7.0,0,53447.0,1.0,1.0,1,18,37376.0,277,...,0,0,0,0,0,0,1,0,0,0


# PCA

In [22]:
df_encoded.drop(["HasDetections"], axis=1, inplace=True)
df_encoded.dropna(inplace = True)

In [24]:
df_encoded.shape

(16100, 771)

In [25]:
df_encoded = df_encoded -df_encoded.mean()
df_encoded.head(1)

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,GeoNameIdentifier,...,Census_ActivationChannel_Retail:TB:Eval,Census_ActivationChannel_Volume:GVLK,Census_ActivationChannel_Volume:MAK,Census_FlightRing_Disabled,Census_FlightRing_NOT_SET,Census_FlightRing_RP,Census_FlightRing_Retail,Census_FlightRing_Unknown,Census_FlightRing_WIF,Census_FlightRing_WIS
0,0.0,0.147764,-0.016522,5470.303913,-0.324348,-0.015901,0.000994,-80.047826,46553.171801,-135.194845,...,0.0,-0.044037,-0.001304,-0.000311,-0.030248,-0.001118,0.058447,-0.023913,-0.001304,-0.001553


In [26]:
pca = PCA(n_components=20)

In [27]:
pca.fit(df_encoded)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [28]:
Xt = pca.transform(df_encoded)

In [29]:
Xt.shape

(16100, 20)

In [33]:
pca.explained_variance_ratio_

array([9.99999934e-01, 5.66300216e-08, 7.23682682e-09, 1.43511942e-09,
       6.80416485e-10, 1.22654483e-10, 5.31522503e-11, 5.34980539e-12,
       3.36101114e-12, 7.36765098e-13, 4.03001012e-13, 1.91583537e-13,
       3.87238243e-14, 3.74137722e-14, 1.45542820e-14, 8.90056571e-15,
       2.73249801e-15, 1.79376231e-15, 1.33944562e-15, 6.74282186e-16])

In [41]:
pca.explained_variance_ratio_[0].sum()

0.999999933831661

In [ ]:
# alternative idea: keep all features, factorize or one-hot everything, then do pca and then neural network.